In [1]:
import requests
import sys
sys.path.append('/Users/gmaurer/GitHub/')
import pyEDIutils.changes as edi
import pandas as pd
import numpy as np

siteabbrev = 'JRN'
scopestr = 'knb-lter-' + siteabbrev.lower()

## Archive a request for earlier data

Do this if needed to update the request archives for the current report .

In [ ]:
#curl -i -X GET "https://pasta.lternet.edu/package/changes/eml?fromDate=2017-02-01T12:00:00&toDate=2020-01-28&scope=knb-lter-jrn"
# This is from the "List recent changes" call  https://pastaplus-core.readthedocs.io/en/latest/doc_tree/pasta_api/data_package_manager_api.html#list-recent-changes
# Only do this if it isn't archived yet!
edi.archive_requested_changes('2023-06-02', '2023-07-15')

In [ ]:
# Get edi activity since a given date
# Note that for some sites this may return more than the API
# can handle in one go and you'll have to go in stages
lter_edi = edi.request_changes('2012-01-01', todt=None, scope=scopestr,
    dedup = True, parsedt=False)

In [ ]:
lter_edi.head()

In [ ]:
# Add method count columns
df = edi.get_counts(lter_edi)
df.head()

In [ ]:
# Count deleted packages
deleted_pkgs = df.loc[df.action=='deleteDataPackage','pkgid']
print(deleted_pkgs)

## Create daily data file

In [ ]:
# Sum to daily methods
daily = edi.counts_to_daily(df)
daily.head()

## Plot activity since 2013


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# Figure config
#font = {'family' : 'normal',
#        'weight' : 'normal',
#        'size'   : 14}

#matplotlib.rc('font', **font)

In [ ]:
# Get a datestamp for saving figures
import datetime
now = datetime.datetime.now().strftime("%Y%m%d")

In [ ]:
# All time plot:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7, 7), sharex=True)

ax[0].plot(daily.index, daily.n_update, label='Update pkg')
ax[0].plot(daily.index, daily.n_create, label='Create pkg')
ax[0].set_ylabel(siteabbrev + ' events')

ax[1].plot(daily.index, daily.n_update.cumsum(), label='Update pkg')
ax[1].plot(daily.index, daily.n_create.cumsum(), label='Create pkg')
ax[1].plot(daily.index, daily.n_tot.cumsum(), label='Total')

plt.ylabel(siteabbrev + ' Cumulative')
fig.autofmt_xdate()
plt.suptitle(siteabbrev + ' activity on EDI (all time)')
plt.legend()

import os
#fig.savefig(os.path.expanduser("~/data/GD_gmaurer@nmsu/IM/figures/JRN_EDI_all_{0}.png".format(now)),
#            dpi=150)

### Total number of packages

In [ ]:
daily.n_tot.sum()

## Plot activity since proposal review

In [ ]:

startdt = "2022-05-30"
daily_subs = edi.counts_to_daily(df[df.date > startdt])
daily_subs.head()

In [ ]:
# Recent plot:
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=(7, 7), sharex=True)

ax[0].plot(daily_subs.index, daily_subs.n_update, label='Update pkg')
ax[0].plot(daily_subs.index, daily_subs.n_create, label='Create pkg')
ax[0].set_ylabel(siteabbrev + ' events')

ax[1].plot(daily_subs.index, daily_subs.n_update.cumsum(), label='Update pkg')
ax[1].plot(daily_subs.index, daily_subs.n_create.cumsum(), label='Create pkg')
ax[1].plot(daily_subs.index, daily_subs.n_tot.cumsum(), label='$Delta$ Total')
#ax[1].axhline(y=63, xmin=0, xmax=1, ls=':', lw=1.5, color='k',
#             label='Proposal goal')

plt.ylabel(siteabbrev + ' Cumulative')
fig.autofmt_xdate()
plt.suptitle(siteabbrev + ' activity on EDI since renewal')
plt.legend()

#fig.savefig(os.path.expanduser(
#    "~/data/GD_gmaurer@nmsu/IM/figures/JRN_EDI_recNoMet_{0}.png".format(now)),
#            dpi=150)

In [ ]:
# Unique packages updated
uniqueUp = df.loc[df.action=='updateDataPackage',:].pkgid.unique()
# Since proposal
recent = df.loc[df.date > startdt,]
uniqueUp_subs = recent.loc[recent.action=='updateDataPackage',:].pkgid.unique()
#rEDI.loc[rEDI.action=='updateDataPackage',].pkgid.unique()
uniqueUp

In [ ]:
pkgtbl = pd.DataFrame({'Total':[daily.n_create.sum(), daily.n_update.sum(),len(uniqueUp)],
                       'Since renewal':[daily_subs.n_create.sum(), daily_subs.n_update.sum(),len(uniqueUp_subs)]})
pkgtbl.index = ['New packages created', 'Package updates', 'Unique package updates']

print(pkgtbl)

# Some comparison totals as of late 2024:
VCR: 344
KBS: 83
BNZ: 669
ARC: 651
SEV: 231
LUQ: 204
HFR: 451
HBR: 310
MCM: 332
PAL: 94
CAP: 265
PIE: 599
JRN: 393
MCR: 112
CCE: 101
